In [ ]:
import import_ipynb
from OpenDataPandaFramework import *

The following parameters should be set to specify how many events will be read in every chunk, the directory for the files you would like to convert into hd5 files and a tag wich will be appended to the output filename to reflect the skimming applied

In [ ]:
# The luminosity we run over
lumi = 10.6
# How many events to read in each round
chunksize = 500000
# How often to write to hd5 file (the smaller, the more files)
printevry = 500000
# Directory containing the input file (must end with .root)
indir   = "/home/michael/Desktop/OpenData/SM_Backgrounds"
#indir   = "/scratch3/eirikgr/openData_13TeV/MC/SM_Backgrounds/"
# The output file name tag to store the skim options used abvove
datatype = indir.split("/")[-1]
# In cases there is a trailing / at the end
if not datatype: datatype = indir.split("/")[-2]
print("INFO \t Data type is {:s}".format(datatype))

# Define if it is signal or background (important for ML classification)
isMC = False
isData = False
isSignal = False
if datatype == "SM_Backgrounds":
    isMC = True
elif datatype == "BSM_Signal_Samples":
    isSignal = True
elif datatype == "Data" or datatype == 'data':
    isData = True
else:
    print("ERROR \t Datatype {:s} is unknown. Setting as background".format(datatype))
    isMC = True
    
nlep = 2
lep_ptcut = [25000,25000]
skimtag = "2L_pt25_25_met50"

Specify the branches to keep in the hd5 files. If you like to check all the availale brances open a file and do events.keys()

In [ ]:
branches = ['eventNumber','channelNumber',
            'mcWeight','scaleFactor_PILEUP','scaleFactor_ELE','scaleFactor_MUON',
            'scaleFactor_BTAG','scaleFactor_LepTRIGGER',
            'met_*','jet_*','lep_*']#'jet_n']#'jet_*']#,'photon_*','fatjet_*','tau_*','lep_*',
            #'ditau_m','truth_*']
#branches = ["*"]

In [ ]:
res = OpenDataPandaFramework(indir,nlep, chunksize, printevry, datatype, skimtag, isMC, isData, isSignal, lep_ptcut, branches, lumi)

In [ ]:
del[res]